In [1]:
import torch as tc
import numpy as np
import Library.BasicFun as bf

from matplotlib import pyplot as plt

print(tc.cuda.is_available())
print(tc.__version__)

True
1.10.0+cu113


In [2]:
# 导入数据
states = np.load('/data/home/scv7454/run/GraduationProject/Data/states.npy', allow_pickle=True)
print(states[0].shape)
data = tc.stack(list(tc.from_numpy(states)))
print(data.shape)

data_mat = data.reshape(1000, -1)
rho = tc.einsum('ab,ac->abc', data_mat.to('cuda:0'), data_mat.conj().to('cuda:0'))
print(rho.shape)
# rho = rho_.reshape(1000*1024, -1)
# print(rho.shape)

# probs = tc.mul(data, data.conj())
# print(probs.shape[0])

(2, 2, 2, 2, 2, 2, 2, 2, 2, 2)
torch.Size([1000, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
torch.Size([1000, 1024, 1024])


In [3]:
import torch as tc
import numpy as np
import Library.BasicFun as bf

from torch.nn import MSELoss, NLLLoss
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from matplotlib import pyplot as plt
from Library.BasicFun import choose_device
from Library.MathFun import series_sin_cos
from Algorithms import ADQC_algo, LSTM_algo

from Library.ADQC import ADQC_LatentGates

def split_time_series(data, length, device=None, dtype=tc.float32):
    samples, targets = list(), list()
    device = choose_device(device)
    for n in range(length, data.shape[0]):
        samples.append(data[n-length:n].clone())
        targets.append(data[n].clone())
    return tc.cat(samples, dim=0).to(device=device, dtype=dtype), tc.stack(targets, dim=0).to(device=device, dtype=dtype)

def fidelity(psi1, psi0):
    f = 0
    for i in range(psi1.shape[0]):
        psi0_ = psi0[i]
        psi1_ = psi1[i]
        x_pos = list(range(len(psi1_.shape)))
        y_pos = x_pos
        f_ = bf.tmul(psi1_.conj(), psi0_, x_pos, y_pos)
        f += tc.sqrt((f_*f_.conj()).real)
    f = f/psi1.shape[0]
    return f

def DataProcess(data, para=None):
    para0 = dict()  # 默认参数
    para0['test_ratio'] = 0.2  # 将部分样本划为测试集
    para0['length_in'] = 10  # 数据样本维数
    para0['length_out'] = 10
    para0['batch_size'] = 200  # 批次大小
    para0['feature_map'] = 'cossin'  # 特征映射
    para0['lattice'] = 'brick'  # ADQC链接形式（brick或stair）
    para0['depth'] = 4  # ADQC层数
    para0['ini_way'] = 'random'  # 线路初始化策略
    para0['lr'] = 2e-3  # 初始学习率
    para0['it_time'] = 1000  # 迭代次数
    para0['print_time'] = 10  # 打印间隔
    para0['device'] = 'cuda:0'
    para0['dtype'] = tc.complex128

    if para is None:
        para = para0
    else:
        para = dict(para0, **para)  # 更新para参数
    para['device'] = bf.choose_device(para['device'])

    num_train = int(data.shape[0] * (1-para['test_ratio']))
    trainset, train_lbs = split_time_series(
        data[:num_train], para['length'], para['device'], para['dtype'])
    testset, test_lbs = split_time_series(
        data[num_train-para['length']:], para['length'], para['device'], para['dtype'])
    trainloader = DataLoader(TensorDataset(trainset, train_lbs), batch_size=para['batch_size'], shuffle=False)
    testloader = DataLoader(TensorDataset(testset, test_lbs), batch_size=para['batch_size'], shuffle=False)
    return trainloader, testloader

def ADQC(para=None):
    para0 = dict()  # 默认参数
    para0['test_ratio'] = 0.2  # 将部分样本划为测试集
    para0['length_in'] = 10  # 数据样本维数
    para0['length_out'] = 10
    para0['batch_size'] = 200  # 批次大小
    para0['feature_map'] = 'cossin'  # 特征映射
    para0['lattice'] = 'brick'  # ADQC链接形式（brick或stair）
    para0['depth'] = 4  # ADQC层数
    para0['ini_way'] = 'random'  # 线路初始化策略
    para0['lr'] = 2e-3  # 初始学习率
    para0['it_time'] = 1000  # 迭代次数
    para0['print_time'] = 10  # 打印间隔
    para0['device'] = 'cuda:0'
    para0['dtype'] = tc.complex128

    if para is None:
        para = para0
    else:
        para = dict(para0, **para)  # 更新para参数
    para['device'] = bf.choose_device(para['device'])

    qc = ADQC_LatentGates(lattice=para['lattice'], num_q=para['length_in'], depth=para['depth'], ini_way=para['ini_way'],
                          device=para['device'], dtype=para['dtype'])
    qc.single_state = False  # 切换至多个态演化模式
    return qc

def train(qc, data, para):
    para0 = dict()  # 默认参数
    para0['test_ratio'] = 0.2  # 将部分样本划为测试集
    para0['length_in'] = 10  # 数据样本维数
    para0['length_out'] = 10
    para0['batch_size'] = 200  # 批次大小
    para0['feature_map'] = 'cossin'  # 特征映射
    para0['lattice'] = 'brick'  # ADQC链接形式（brick或stair）
    para0['depth'] = 4  # ADQC层数
    para0['ini_way'] = 'random'  # 线路初始化策略
    para0['lr'] = 2e-3  # 初始学习率
    para0['it_time'] = 1000  # 迭代次数
    para0['print_time'] = 10  # 打印间隔
    para0['device'] = 'cuda:0'
    para0['dtype'] = tc.complex128
    if para is None:
        para = para0
    else:
        para = dict(para0, **para)  # 更新para参数

    optimizer = Adam(qc.parameters(), lr=para['lr'])

    num_train = int(data.shape[0] * (1-para['test_ratio']))
    trainset, train_lbs = split_time_series(
        data[:num_train], para['length'], para['device'], para['dtype'])
    testset, test_lbs = split_time_series(
        data[num_train-para['length']:], para['length'], para['device'], para['dtype'])
    trainloader = DataLoader(TensorDataset(trainset, train_lbs), batch_size=para['batch_size'], shuffle=False)
    testloader = DataLoader(TensorDataset(testset, test_lbs), batch_size=para['batch_size'], shuffle=False)

    loss_train_rec = list()
    loss_test_rec = list()

    for t in range(para['it_time']):
        loss_tmp = 0.0
        for n, (samples, lbs) in enumerate(trainloader):
            psi0 = samples
            psi1 = qc(psi0)
            loss = (0.5*(psi1 - lbs)**2).mean(dtype=tc.complex64)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_tmp += loss.item() * samples.shape[0]

        if (t+1) % para['print_time'] == 0:
            loss_train_rec.append(loss_tmp / (1-para0['test_ratio']))
            loss_tmp = 0.0
            with tc.no_grad():
                for n, (samples, lbs) in enumerate(testloader):
                    psi0 = samples
                    psi1 = qc(psi0)
                    loss = (0.5*(psi1 - lbs)**2).mean(dtype=tc.complex64)
                    loss_tmp += loss.item() * samples.shape[0]
            loss_test_rec.append(loss_tmp / para0['test_ratio'])
            print('Epoch %i: train loss %g, test loss %g' %
                (t+1, loss_train_rec[-1].real, loss_test_rec[-1].real))

    with tc.no_grad():
        results = dict()
        psi0 = trainset
        psi1 = qc(psi0)
        output = psi1
        output = tc.cat([data[:para['length']].to(dtype=output.dtype), output.to(device=data.device)], dim=0)
        results['train_pred'] = output.data
        psi0 = testset
        psi1 = qc(psi0)
        output1 = psi1
        output1 = output1.data.to(device=data.device)
        results['test_pred'] = output1
        results['train_loss'] = loss_train_rec
        results['test_loss'] = loss_test_rec
    return qc, results, para

In [4]:
# 导入模型进行训练
print('设置参数')
# 通用参数
para = {'lr': 1e-2,  # 初始学习率
        'length_tot': 500,  # 序列总长度
        'order': 10,  # 生成序列的傅里叶阶数
        'length': 1,  # 每个样本长度
        'batch_size': 1000,  # batch大小
        'it_time': 200,  # 总迭代次数
        'dtype': tc.complex128,  # 数据精度
        'device': choose_device()}  # 计算设备（cuda优先）

# ADQC参数
para_adqc = {'depth': 4}  # ADQC量子门层数

para_adqc = dict(para, **para_adqc)
qc = ADQC(para_adqc)

for i in range(rho.shape[1]):
    print(i)
    data = rho[:,:,i]
    data = data.reshape(1000, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2)
#     print(data[20])
    qc, results_adqc, para_adqc = train(qc, data, para_adqc)
    output_adqc = tc.cat([results_adqc['train_pred'],
                    results_adqc['test_pred']], dim=0)

设置参数
0
Epoch 10: train loss -0.000174618, test loss 9.3284e-05
Epoch 20: train loss -0.000286222, test loss 8.78416e-05
Epoch 30: train loss -0.00036313, test loss 8.62353e-05
Epoch 40: train loss -0.000414489, test loss 7.96192e-05
Epoch 50: train loss -0.000453541, test loss 6.97629e-05
Epoch 60: train loss -0.00048593, test loss 6.21932e-05
Epoch 70: train loss -0.000511483, test loss 5.96102e-05
Epoch 80: train loss -0.000533493, test loss 6.22148e-05
Epoch 90: train loss -0.000553681, test loss 6.64251e-05
Epoch 100: train loss -0.000572845, test loss 7.05443e-05
Epoch 110: train loss -0.000591076, test loss 7.44407e-05
Epoch 120: train loss -0.000607982, test loss 7.81238e-05
Epoch 130: train loss -0.000623364, test loss 8.15421e-05
Epoch 140: train loss -0.000637439, test loss 8.52356e-05
Epoch 150: train loss -0.000650545, test loss 8.89727e-05
Epoch 160: train loss -0.00066292, test loss 9.24244e-05
Epoch 170: train loss -0.000674684, test loss 9.5468e-05
Epoch 180: train loss

In [1]:
%config InlineBackend.figure_format="svg"
# 画图
x_range = rho.shape[0]
x = list(range(rho.shape[0]))
train_num = int(x_range - x_range*para_adqc['test_ratio'])
rho0_real = data[:, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0].cpu()
rho0_pred = output_adqc[:, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0].cpu()
plt.plot(x, rho0_real)
plt.plot(x[:train_num], rho0_pred[:train_num], marker='')
plt.plot(x[train_num:], rho0_pred[train_num:], marker='')

tc.save(qc, 'GraduationProject/Data/qc_rho.pth')
np.save('GraduationProject/Data/rho_adqc', output_adqc.cpu())

NameError: name 'rho' is not defined